In [25]:
#начинаем с импорта библиотек
import requests
from requests.auth import HTTPProxyAuth # для проксей
from  bs4 import BeautifulSoup as bs
import csv
from urllib.request import urljoin
from tqdm.notebook import tqdm
from collections import deque
import pandas as pd
from datetime import datetime
import math

In [2]:
# Задаем глобальные параметры
headers = {'accept': '*/*',
           'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36'
          }
domen = 'https://www.decathlon.ru'
url='https://www.decathlon.ru/C-109639-sport-ot-a-do-ja'
file_product_links = 'decathlon_product_links.csv'
file_products = 'decathlon_products.csv'

In [3]:
# Функция сбора ссылок на основные продуктовые категории
def get_category_links(headers, domen, url):
    session = requests.Session()
    r = session.get(url, headers=headers)
    soup = bs(r.text, 'lxml')
    links = []
    cat_links=[]
    urls = set()
    ul = soup.find('div', class_='alphabet_recherche').find_all('a', href=True)
    for u in ul:
        urls.add(u['href'])
    for i in urls:
        link  = urljoin(domen,i)
        links.append(link)
    return links

In [37]:
#  блок сбора урлов пагинации
def get_pagination(url, headers):
    session = requests.Session()
    r = session.get(url, headers=headers)
    pagi=[]
    pagi.append(url)
    if r.status_code == 200:
        soup = bs(r.content, 'lxml')
        ppage = soup.find('span', class_='filter-results-number').text
        num=math.ceil(int(ppage)/40)
        for i in range(2,num+1):
            u = str(url)+f'#page{i}'
            pagi.append(u)
    return pagi

In [125]:
# Фуекция сбора инфы с грида 
def get_all_params(grid_links, headers, domen):
    params={}
    params['title']=[]
    params['price']=[]
    params['item_url']=[]
    params['item_img']=[]
    params['brand']=[]
    params['num_otzyv']=[]
    for i in tqdm(grid_links, desc='Getting info from grid', leave=False):
        display(i)
        session = requests.Session()
        r = session.get(i, headers=headers)
        soup = bs(r.text, 'lxml')
        li = soup.find_all('li', class_='new-product-thumbnail desktop')
        for i in li:
            title = i.find('h3', class_='product-label').text
            price = i.find('div', class_='price').text.strip().replace('\xa0','')
            link = domen+str(i.find('a', href=True)['href'])
            brand = i.find('p', class_='product-brand').text
            img = i.find('img', attrs={'data-src':True})['data-src']
            try:
                otzyv = i.find('div', class_='product-rating-number').text.strip()
            except AttributeError:
                pass
            params['title'].append(title)
            params['price'].append(price)
            params['item_url'].append(link)
            params['item_img'].append(img)
            params['brand'].append(brand)
            params['num_otzyv'].append(otzyv) 
    return params   

Запускаем весь парсер

In [129]:
#Собираем список категорий
cl=get_category_links(headers, domen, url)
#Запускаем пагинатор
temp_list = []
for i in catlist:
    dl = get_pagination(i, headers)
    temp_list.append(dl)
#выпрямляем список пагинационных урлов    
grid_links = []
for sublist in temp_list:
    for item in sublist:
        grid_links.append(item)
# Запуск парсера
pl = get_all_params(grid_links, headers, domen)

'https://www.decathlon.ru/C-257793-sportivnaya-gimnastica'

'https://www.decathlon.ru/C-257793-sportivnaya-gimnastica#page2'

'https://www.decathlon.ru/C-96730-tancy-i-gimnastika'

'https://www.decathlon.ru/C-96730-tancy-i-gimnastika#page2'

'https://www.decathlon.ru/C-96730-tancy-i-gimnastika#page3'

...

'https://www.decathlon.ru/C-96758-beg#page9'

'https://www.decathlon.ru/C-96758-beg#page10'

'https://www.decathlon.ru/C-96758-beg#page11'

'https://www.decathlon.ru/C-96758-beg#page12'

'https://www.decathlon.ru/C-96758-beg#page13'

In [127]:
# Загружаем итоги в датафрейм
df = pd.DataFrame(pl)
df.sample(35)

,title,price,item_url,item_img,brand,num_otzyv
643,ПОВЯЗКА ДЛЯ БЕГА УНИВЕРСАЛЬНАЯ С ПРИНТОМ СЕРАЯ,299руб.,https://www.decathlon.ru/povzka-universalna-se...,https://contents.mediadecathlon.com/p1807609/k...,KALENJI,(4028)
56,Леггинсы для художественной гимнастики женские...,699руб.,https://www.decathlon.ru/legginsy-d-hud-gimn-z...,https://contents.mediadecathlon.com/p1684847/k...,DOMYOS,(200)
307,КУРТКА НЕПРОМОКАЕМАЯ ДЛЯ ТРЕЙЛРАННИНГА МУЖСКАЯ...,4999руб.,https://www.decathlon.ru/kurtka-trejl-muzhska-...,https://contents.mediadecathlon.com/p1762401/k...,EVADICT,(791)
73,Купальник с длинными рукавами для художественн...,2499руб.,https://www.decathlon.ru/kupalnik-900-zhen-sin...,https://contents.mediadecathlon.com/p1712258/k...,DOMYOS,(71)
64,НОСКИ ДЛЯ ИГРЫ В БОЛЬШОЙ ТЕННИС С ВЫСОКОЙ МАНЖ...,79руб.,https://www.decathlon.ru/noski-vysokie-rs-130-...,https://contents.mediadecathlon.com/p1841848/k...,ARTENGO,(29)
448,УТЕПЛЕННЫЕ БРЮКИ ДЛЯ БЕГА ЖЕНСКИЕ RUN WARM ЧЕРНЫЕ,1499руб.,https://www.decathlon.ru/tepl-brjuki-run-warm-...,https://contents.mediadecathlon.com/p1932757/k...,KALENJI,(64)
190,Шорты для современных танцев облегающие детские,399руб.,https://www.decathlon.ru/shorty-det-chernye-id...,https://contents.mediadecathlon.com/p1704397/k...,DOMYOS,(476)
30,НАКЛАДКИ ДЛЯ ХУДОЖЕСТВЕННОЙ ГИМНАСТИКИ НА ПЕРЕ...,1699руб.,https://www.decathlon.ru/nakladki-gammbf-900-i...,https://contents.mediadecathlon.com/p1560851/k...,DOMYOS,(12)
333,ГОЛЬФЫ ТОНКИЕ ВЫСОКИЕ ИЗ ШЕРСТИ МЕРИНОСА ДЛЯ Б...,799руб.,https://www.decathlon.ru/golfy-utepln-vysokie-...,https://contents.mediadecathlon.com/p1961541/k...,KIPRUN,(242)
616,ДЛИННЫЕ ТАЙТСЫ ДЛЯ БЕГА KIPRUN WARM МУЖСКИЕ,2499руб.,https://www.decathlon.ru/dlin-tajtsy-kiprun-wa...,https://contents.mediadecathlon.com/p1886339/k...,KIPRUN,(155)


In [128]:
# Сколько позиций спарсили
df.shape[0]

720